In [1]:
import jieba
import random
import numpy as np
with open('rating_data.txt')as f:
    data=f.readlines()

sent=[]
for x in data:
    if len(x.split('\t'))==7:
        sent.append(list(jieba.cut(x.replace('<sssss>','').replace('\n','').split('\t')[6])))

sentchar=[x.replace('<sssss>','').replace('\n','').split('\t')[6] for x in data if len(x.split('\t'))==7]
label=np.asarray([x.replace('<sssss>','').split('\t')[:6] for x in data if len(x.split('\t'))==7]）


word_dict={'padding':[0,999999]}
for i in sent:
    for j in i:
        if j not in word_dict:
            word_dict[j]=[len(word_dict),1]
        else:
            word_dict[j][1]+=1

char_dict={'padding':0}
for i in sentchar:
    for j in i:
        if j not in char_dict:
            char_dict[j]=len(char_dict)


MAX_SENT_LENGTH=750
MAX_SENT_LENGTH_CHAR=1500
                 
idsent=[]
for i in sent:
    tokens=[word_dict[j][0] for j in i if word_dict[j][0]>=10]
    tokens=tokens[:MAX_SENT_LENGTH]
    tokens+=[0]*(MAX_SENT_LENGTH-len(tokens))
    idsent.append(tokens)

idsentchar=[]
for i in sentchar:
    tokens=[char_dict[j] for j in i]
    tokens=tokens[:MAX_SENT_LENGTH_CHAR]
    tokens+=[0]*(MAX_SENT_LENGTH_CHAR-len(tokens))
    idsentchar.append(tokens)

all_data=np.array(idsent,dtype='int32')
all_data_char=np.array(idsentchar,dtype='int32')

import keras
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from sklearn.metrics import *



class AttLayer(Layer):

    def __init__(self, **kwargs): 
        self.init = initializers.get('normal') 
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], 1)))
        self.b = K.variable(self.init(( 1,))) 
        self.trainable_weights = [self.W,self.b]
        super(AttLayer, self).build(input_shape)   

    def call(self, x, mask=None):
 
        eij = K.tanh(K.dot(x, self.W)+self.b)
        ai = K.exp(eij)
 
        weights = ai / K.expand_dims(K.sum(ai, axis=1), 1)
        print('weights', weights.shape)
        weighted_input = x * weights 
        return K.sum(weighted_input, axis=1)
                 
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]





index=list(range(len(label)))
random.shuffle(index)

train_reviews=all_data[index[:int(len(label)*0.8)]]
val_reviews=all_data[index[int(len(label)*0.8):int(len(label)*0.9)]]
test_reviews=all_data[index[int(len(label)*0.9):]]
                 
train_reviews_char=all_data_char[index[:int(len(label)*0.8)]]
val_reviews_char=all_data_char[index[int(len(label)*0.8):int(len(label)*0.9)]]
test_reviews_char=all_data_char[index[int(len(label)*0.9):]]
                 
train_label=label[index[:int(len(label)*0.8)]]
val_label=label[index[int(len(label)*0.8):int(len(label)*0.9)]]
test_label=label[index[int(len(label)*0.9):]]

for repeat in range(10):
    keras.backend.clear_session()
    embedding_layer = Embedding(len(word_dict),200,input_length=MAX_SENT_LENGTH, trainable=True)
    embedding_layer_char = Embedding(len(char_dict),100,input_length=MAX_SENT_LENGTH_CHAR, trainable=True)
    
    review_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    review_embedded_sequences = embedding_layer(review_input)
    d_review_emb=Dropout(0.2)(review_embedded_sequences)
    
    review_input_char = Input(shape=(MAX_SENT_LENGTH_CHAR,), dtype='int32')
    review_embedded_sequences_char = embedding_layer_char(review_input_char)
    d_review_emb_char=Dropout(0.2)(review_embedded_sequences_char)    
    
    
    
    review_w_cnn = Convolution1D(nb_filter=200, filter_length=3, padding='same', activation='relu', strides=1)(d_review_emb)
    review_d_w_cnn = Dropout(0.2)(review_w_cnn)
    
    
    review_c_cnn = Convolution1D(nb_filter=100, filter_length=3, padding='same', activation='relu', strides=1)(d_review_emb_char)
    review_d_c_cnn = Dropout(0.2)(review_c_cnn)
    
    
    review_w_att1 = AttLayer()(review_d_w_cnn)
    review_c_att1 = AttLayer()(review_d_c_cnn)
    review_att1=concatenate([review_w_att1,review_c_att1])
    
    
    review_w_att2 = AttLayer()(review_d_w_cnn)
    review_c_att2 = AttLayer()(review_d_c_cnn)
    review_att2=concatenate([review_w_att2,review_c_att2])
    aspect_score1=Dense(1)(review_att2)
    
    review_w_att3 = AttLayer()(review_d_w_cnn)
    review_c_att3 = AttLayer()(review_d_c_cnn)
    review_att3=concatenate([review_w_att3,review_c_att3])
    aspect_score2=Dense(1)(review_att3)
    
    review_w_att4 = AttLayer()(review_d_w_cnn)
    review_c_att4 = AttLayer()(review_d_c_cnn)
    review_att4=concatenate([review_w_att4,review_c_att4])
    aspect_score3=Dense(1)(review_att4)
    
    review_w_att5 = AttLayer()(review_d_w_cnn)
    review_c_att5 = AttLayer()(review_d_c_cnn)
    review_att5=concatenate([review_w_att5,review_c_att5])
    aspect_score4=Dense(1)(review_att5)
    
    review_w_att6 = AttLayer()(review_d_w_cnn)
    review_c_att6 = AttLayer()(review_d_c_cnn)
    review_att6=concatenate([review_w_att6,review_c_att6])
    aspect_score5=Dense(1)(review_att6)
    
    
    overall_rep = concatenate([review_att1,review_att2,review_att3,review_att4,review_att5,review_att6])
    overall_score=Dense(1)(overall_rep)
    
    aspect_score = concatenate([aspect_score1,aspect_score2,aspect_score3,aspect_score4,aspect_score5])



    model = Model([review_input,review_input_char], [overall_score,aspect_score])
    
    model.compile(loss='mae', optimizer='adam', metrics=['acc'],loss_weights=[0.5,0.5])
    
    for p in range(10):
        model.fit([train_reviews,train_reviews_char],[train_label[:,:1],train_label[:,1:]],shuffle=True, batch_size=50, epochs=1,verbose=1)
        
        y_pred = model.predict([val_reviews,val_reviews_char], batch_size=50, verbose=1)[0]
        result=np.mean(np.abs(y_pred-val_label),axis=0)
        overall=np.round(np.minimum(np.maximum(y_pred,1),5))
        print(result,accuracy_score(overall,val_label[:,:1]))#MAE,Acc
        
        y_pred = model.predict([test_reviews,test_reviews_char], batch_size=50, verbose=1)[0]
        result=np.mean(np.abs(y_pred-test_label),axis=0)
        overall=np.round(np.minimum(np.maximum(y_pred,1),5))
        print(result,accuracy_score(overall,test_label[:,:1]))
        #report the test results according to the best validation results
